In [1]:
import numpy as np, pandas as pd
from tqdm import tqdm_notebook, tqdm
tqdm.pandas(desc="my bar!")

In [2]:
def extr_data_txt(path):
    """
    Load data from text file.
    """
    with open(path, "r") as f:
        data = []
        for itr, line in tqdm_notebook(enumerate(f)):
            # Because we got annotation in the first two lines
            if itr >= 2:
                data.append(line.split())
    return data

In [3]:
train_test_valid_anot = pd.DataFrame(extr_data_txt('data/Eval/list_eval_partition.txt'), 
                                     columns=['Path', 'type'])
train_test_valid_anot.to_csv('data/Anno/train_test_valid_anot.csv' ,index=False)

In [5]:
categories_img = pd.DataFrame(extr_data_txt('data/Anno/list_category_img.txt'), 
                          columns=['Path', 'cat'])
categories_img['cat'] = categories_img['cat'].apply(lambda x: int(x)-1)
categories_img.to_csv('data/Anno/categories_img.csv' ,index=False)

In [6]:
box_img = pd.DataFrame(extr_data_txt('data/Anno/list_bbox.txt'), 
                          columns=['Path', 'x1', 'y1', 'x2', 'y2'])
box_img.to_csv('data/Anno/bbox_img.csv' ,index=False)

In [7]:
bbox_img = pd.read_csv('data/Anno/bbox_img.csv')

In [8]:
import cv2

In [9]:
def convert_labels(path, x1, y1, x2, y2):
    """
    Definition: Parses label files to extract label and bounding box
        coordinates.  Converts (x1, y1, x1, y2) KITTI format to
        (x, y, width, height) normalized YOLO format.
    """
    size = get_img_shape(path)
    dw = 1./size[0]
    dh = 1./size[1]
    x = (x1 + x2)/2.0
    y = (y1 + y2)/2.0
    w = x2 - x1
    h = y1 - y2
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

def get_img_shape(path):
    path = 'data/'+path
    img = cv2.imread(path)
    try:
        return img.shape
    except AttributeError:
        print('error! ', path)
        return (None, None, None)

In [10]:
bbox_img['x'], bbox_img['y'], bbox_img['width'], bbox_img['height'] = zip(*bbox_img.progress_apply(lambda row: convert_labels(row['Path'], row['x1'], row['y1'],
                                                                                                                              row['x2'], row['y2']), axis=1))

my bar!: 100%|██████████| 289222/289222 [02:20<00:00, 2055.87it/s]


In [11]:
df = bbox_img.merge(train_test_valid_anot).merge(categories_img)

In [12]:
df.to_csv('data/Anno/annotation_w-o_atr.csv', index=False)

In [3]:
dtypes = {str(i): np.int8 for i in range(0, 1000)}
mem = pd.read_csv('data/Anno/Atr_img.csv', dtype=dtypes)

In [4]:
mem['Path'] = mem['1000']
del mem['1000']

In [5]:
with open('data/Anno/list_attr_cloth.txt', "r") as f:
    data = []
    for itr, line in tqdm_notebook(enumerate(f)):
        if itr >= 2:
                line = line.split()
                data.append(line[0])
# Because last column is 'Path'
data.append('Path')

In [6]:
mem.columns = data
mem.to_csv('data/Anno/annotation_atr.csv', index=False)

In attribute labels, "1" represents positive while "-1" represents negative, '0' represents unknown;

Landmarks 

In [13]:
list_ladmarks = pd.DataFrame(extr_data_txt('data/Anno/list_landmarks.txt'))

land_columns = 'image_name  clothes_type  variation_type  landmark_visibility_1 landmark_location_x_1 landmark_location_y_1  landmark_visibility_2 landmark_location_x_2 landmark_location_y_2  landmark_visibility_3 landmark_location_x_3 landmark_location_y_3  landmark_visibility_4 landmark_location_x_4 landmark_location_y_4  landmark_visibility_5 landmark_location_x_5 landmark_location_y_5  landmark_visibility_6 landmark_location_x_6 landmark_location_y_6  landmark_visibility_7 landmark_location_x_7 landmark_location_y_7  landmark_visibility_8 landmark_location_x_8 landmark_location_y_8'.split()
list_ladmarks.rename(columns={i: land_columns[i] for i in range(0, 26)}, inplace=True)

list_ladmarks.to_csv('data/Anno/list_landmarks.csv' ,index=False)

In [14]:
list_ladmarks = pd.read_csv('data/Anno/list_landmarks.csv')